In [23]:
from sklearn import *
import lightgbm as lgbm
import xgboost as xgb
import catboost as cat
from sklearn.model_selection import train_test_split
from sqlalchemy import column
import pandas as pd
import numpy as np
#!pip install optuna 
import optuna

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import BayesianRidge

from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [24]:
def convert_to_category(X):
    for col in X.columns:
        if ( (X[col].dtype != 'int64') and (X[col].dtype != 'float64') and (X[col].dtype != 'bool')):
            X[col] = X[col].astype('category')
            
    return X

def remove_outliers(df, column_names, n):
    for column_name in column_names:
        mean = df[column_name].mean()
        std = df[column_name].std()
        df = df[(df[column_name] > mean - n*std) & (df[column_name] < mean + n*std)]
    return df
    
def log_func(y, shift_var):
    return np.log(y+shift_var)

def exp_func(y, shift_var):
    return np.exp(y)-shift_var

In [25]:
stores_train = pd.read_csv('data/stores_train.csv')
stores_test = pd.read_csv('data/stores_test.csv')

plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')
grunnkrets = pd.read_csv('data/grunnkrets_norway_stripped.csv')
grunnkrets_ages = pd.read_csv('data/grunnkrets_age_distribution.csv')
grunnkrets_household_types = pd.read_csv('data/grunnkrets_households_num_persons.csv')
grunnkrets_household_income = pd.read_csv('data/grunnkrets_income_households.csv')
busstops = pd.read_csv('data/busstops_norway.csv')

stores_train_num_busstops = pd.read_csv('data/other_data/stores_train_num_closest_busstop.csv')
stores_test_num_busstops = pd.read_csv('data/other_data/stores_test_num_closest_busstop.csv')

stores_train = pd.merge(stores_train, stores_train_num_busstops, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_busstops, on='store_id', how='left')

stores_train_closest_busstop = pd.read_csv('data/other_data/stores_train_closest_busstop.csv')
stores_test_closest_busstop = pd.read_csv('data/other_data/stores_test_closest_busstop.csv')

stores_train = pd.merge(stores_train, stores_train_closest_busstop, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_busstop, on='store_id', how='left')

stores_train_closest_store_lv1 = pd.read_csv('data/other_data/stores_train_closest_store_lv1.csv')
stores_test_closest_store_lv1 = pd.read_csv('data/other_data/stores_test_closest_store_lv1.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv1, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv1, on='store_id', how='left')

stores_train_closest_store_lv2 = pd.read_csv('data/other_data/stores_train_closest_store_lv2.csv')
stores_test_closest_store_lv2 = pd.read_csv('data/other_data/stores_test_closest_store_lv2.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv2, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv2, on='store_id', how='left')

stores_train_closest_store_lv3 = pd.read_csv('data/other_data/stores_train_closest_store_lv3.csv')
stores_test_closest_store_lv3 = pd.read_csv('data/other_data/stores_test_closest_store_lv3.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv3, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv3, on='store_id', how='left')

stores_train_closest_store_lv4 = pd.read_csv('data/other_data/stores_train_closest_store_lv4.csv')
stores_test_closest_store_lv4 = pd.read_csv('data/other_data/stores_test_closest_store_lv4.csv')

stores_train = pd.merge(stores_train, stores_train_closest_store_lv4, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_closest_store_lv4, on='store_id', how='left')

stores_train_num_closest_stores_lv1 = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv1.csv')
stores_test_num_closest_stores_lv1 = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv1.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv1, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv1, on='store_id', how='left')

stores_train_num_closest_stores_lv2 = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv2.csv')
stores_test_num_closest_stores_lv2 = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv2.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv2, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv2, on='store_id', how='left')

stores_train_num_closest_stores_lv3 = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv3.csv')
stores_test_num_closest_stores_lv3 = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv3.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv3, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv3, on='store_id', how='left')

stores_train_num_closest_stores_lv4 = pd.read_csv('data/other_data/stores_train_num_closest_stores_lv4.csv')
stores_test_num_closest_stores_lv4 = pd.read_csv('data/other_data/stores_test_num_closest_stores_lv4.csv')

stores_train = pd.merge(stores_train, stores_train_num_closest_stores_lv4, on='store_id', how='left')
stores_test = pd.merge(stores_test, stores_test_num_closest_stores_lv4, on='store_id', how='left')

stores_train_copy= stores_train.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns='sales_channel_name', inplace=True)
plaace_hierarchy_copy['lv1'] = plaace_hierarchy_copy['lv1'].astype('category')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('category')


grunnkrets_copy = grunnkrets.copy()
grunnkrets_copy.rename(columns={'year': 'year_1'}, inplace=True)

grunnkrets_ages_copy = grunnkrets_ages.copy()
grunnkrets_ages_copy.rename(columns={'year': 'year_2'}, inplace=True)

grunnkrets_household_types_copy = grunnkrets_household_types.copy()
grunnkrets_household_types_copy.rename(columns={'year': 'year_3'}, inplace=True)
grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]] = grunnkrets_household_types_copy[grunnkrets_household_types_copy.columns[2:]].astype('int64')

grunnkrets_household_income_copy = grunnkrets_household_income.copy()
grunnkrets_household_income_copy.rename(columns={'year': 'year_4', 'singles': 'singles_income','couple_without_children':'couple_without_children_income'}, inplace=True)
grunnkrets_household_income_copy.rename(columns={'singles': 'singles_income', }, inplace=True)

# set the values that are 0 to the lowest value in the column
# for column in grunnkrets_household_income_copy.columns[2:]:
#     grunnkrets_household_income_copy[column] = grunnkrets_household_income_copy[column].apply(lambda x: grunnkrets_household_income_copy[column].min() if x == 0 else x)
    


df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, on='plaace_hierarchy_id', how='left')
df = pd.merge(df, grunnkrets_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_ages_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_types_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_income_copy, on='grunnkrets_id', how='left')

df.drop_duplicates(subset=['store_id'], keep='first', inplace=True)

columns_to_drop = ['revenue',
                  'store_id',
                  'plaace_hierarchy_id',
                  'grunnkrets_id',
                  'geometry',
                  'year',
                  'year_1',
                  'year_2',
                  'year_3',
                  'year_4',
                  'address',
                  'store_name',
                  ]

X = df.drop(columns=columns_to_drop)
X = convert_to_category(X)


categorical_features = list(X.select_dtypes(include=['category']).columns)


# fill in missing values of float columns with mean

for col in X.columns:
    if ( (X[col].dtype == 'float64') or (X[col].dtype == 'int64')):
        X[col].fillna(X[col].mean(), inplace=True)
        
        
columns_to_scale_scaler = ['all_households', 'singles_income', 'couple_without_children_income', 'couple_with_children', 'other_households', 'single_parent_with_children',]
        
columns_to_scale_log = [
    'lat', 'lon', 
    'area_km2',
                     
        #'all_households', 'singles_income', 'couple_without_children_income', 'couple_with_children', 'other_households', 'single_parent_with_children',
                     
        'couple_children_0_to_5_years', 'couple_children_18_or_above', 'couple_children_6_to_17_years', 'couple_without_children', 'single_parent_children_0_to_5_years', 'single_parent_children_18_or_above', 'single_parent_children_6_to_17_years', 'singles',
                     
        'age_0', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8', 'age_9', 'age_10', 'age_11',
        'age_12', 'age_13', 'age_14', 'age_15', 'age_16', 'age_17', 'age_18',
        'age_19', 'age_20', 'age_21', 'age_22', 'age_23', 'age_24', 'age_25',
        'age_26', 'age_27', 'age_28', 'age_29', 'age_30', 'age_31', 'age_32',
        'age_33', 'age_34', 'age_35', 'age_36', 'age_37', 'age_38', 'age_39',
        'age_40', 'age_41', 'age_42', 'age_43', 'age_44', 'age_45', 'age_46',
        'age_47', 'age_48', 'age_49', 'age_50', 'age_51', 'age_52', 'age_53',
        'age_54', 'age_55', 'age_56', 'age_57', 'age_58', 'age_59', 'age_60',
        'age_61', 'age_62', 'age_63', 'age_64', 'age_65', 'age_66', 'age_67',
        'age_68', 'age_69', 'age_70', 'age_71', 'age_72', 'age_73', 'age_74',
        'age_75', 'age_76', 'age_77', 'age_78', 'age_79', 'age_80', 'age_81',
        'age_82', 'age_83', 'age_84', 'age_85', 'age_86', 'age_87', 'age_88',
        'age_89', 'age_90',
                     
                     
        'num_closest_busstops',
        'closest_busstop', 'closest_store_lv1', 'closest_store_lv2',
        'closest_store_lv3', 'closest_store_lv4', 'num_closest_stores_lv1',
        'num_closest_stores_lv2', 'num_closest_stores_lv3',
        'num_closest_stores_lv4'
                    
                     
                     ]



scaler = preprocessing.StandardScaler()

scaler.fit(X[columns_to_scale_scaler])

X[columns_to_scale_scaler] = scaler.transform(X[columns_to_scale_scaler])

for columns in columns_to_scale_log:
    X[columns] = np.log1p(X[columns])



full_pipeline = ColumnTransformer([('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)], remainder='passthrough')

encoder = full_pipeline.fit(X)
X_encoded = encoder.transform(X)

y = df.revenue
y = np.log1p(y)


In [26]:
X

,sales_channel_name,lat,lon,chain_name,mall_name,num_closest_busstops,closest_busstop,closest_store_lv1,closest_store_lv2,closest_store_lv3,...,single_parent_children_0_to_5_years,single_parent_children_18_or_above,single_parent_children_6_to_17_years,singles,all_households,singles_income,couple_without_children_income,couple_with_children,other_households,single_parent_with_children
0,Hamburger restaurants,4.106654,2.416354,MCDONALDS,Magasinet Drammen,6.113682,6.207893,4.125997,4.237666,7.875765,...,0.000000,0.000000,2.197225,4.369448,-0.740345,-0.058520,0.927876,0.483401,-0.331773,-0.357127
16,Hamburger restaurants,4.109459,2.462493,MCDONALDS,NaN,7.075809,4.638176,0.022038,3.510934,4.663170,...,0.000000,0.000000,0.000000,1.791759,-1.044591,0.474519,0.413948,0.151940,-0.917482,-0.760925
24,Hamburger restaurants,4.113571,2.420782,BURGER KING,Kuben Hønefoss,5.337538,3.278344,4.155566,4.278054,10.429733,...,2.639057,1.945910,2.564949,5.017280,-1.302371,-1.150184,-0.348082,-0.575285,-0.622551,-0.642833
40,Hamburger restaurants,4.223671,2.733055,BURGER KING,Glasshuspassasjen,5.164786,3.972048,3.679474,3.679474,7.580426,...,2.484907,2.197225,2.772589,5.541264,-0.792344,0.628035,-0.153144,-0.557386,-0.193308,-0.538709
56,Hamburger restaurants,4.164462,2.431403,BURGER KING,Tillertorget,6.501290,5.118987,0.040295,4.861209,5.157808,...,0.000000,0.000000,0.000000,1.386294,1.659326,0.653621,0.478391,0.064434,0.400017,0.488561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197996,Beer and soda shop,4.101259,2.440691,NaN,NaN,4.983607,4.358595,3.376449,4.953150,5.781439,...,3.688879,3.713572,3.931826,6.368187,-0.920680,-0.873004,-1.100445,-0.520262,-0.793556,-0.550137
198012,Beer and soda shop,4.123903,2.459045,GULATING GRUPPEN,CC Gjøvik,5.976351,5.787584,0.019726,0.019726,0.019726,...,1.791759,2.397895,2.639057,5.575949,-1.920820,-1.521179,-2.476288,-1.952835,-1.729582,-0.487917
198028,Beer and soda shop,4.109997,2.485545,GULATING GRUPPEN,Strømmen Storsenter,6.298949,4.665844,4.092969,4.127636,4.127636,...,3.931826,3.433987,3.526361,6.011267,-0.335421,0.141903,-0.227252,-0.078094,-0.118537,-0.236496
198044,Beer and soda shop,4.091846,1.907418,NaN,Bystasjonen,6.437752,4.689210,3.394390,3.466244,5.220442,...,0.000000,1.609438,0.000000,4.465908,-0.947233,0.436141,-0.182143,-0.955802,-0.515932,-0.767274


### LGBM

In [33]:
def objective(trial,data=X,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param = {
        'metric': 'rmse', 
        'random_state': 48,
        #'n_estimators': trial.suggest_int('n_estimators', 100, 20000),
        'n_estimators': 20000,
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = lgbm.LGBMRegressor(**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmsle = mean_squared_log_error(test_y, preds,squared=False)
    
    return rmsle

In [34]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2022-11-03 20:57:40,420] A new study created in memory with name: no-name-1969fb40-db0e-4696-88a1-59e953f043dc
c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
c:\Users\levit\.pyenv\pyenv-win\versions\3.10.7\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Re

Number of finished trials: 50
Best trial: {'reg_alpha': 0.04913102153584967, 'reg_lambda': 5.833548843574208, 'colsample_bytree': 0.7, 'subsample': 0.4, 'learning_rate': 0.014, 'max_depth': 10, 'num_leaves': 422, 'min_child_samples': 251, 'min_data_per_groups': 1}


In [19]:
params = study.best_trial.params

params

{'reg_alpha': 0.001435512306542802,
 'reg_lambda': 0.0012165641231410146,
 'colsample_bytree': 0.3,
 'subsample': 0.7,
 'learning_rate': 0.01,
 'max_depth': 10,
 'num_leaves': 745,
 'min_child_samples': 225,
 'min_data_per_groups': 2}

### XGBoost

In [ ]:
def objective(trial,data=X_encoded,target=y):
    
    train_x, test_x, train_y, test_y = train_test_split(data, target, test_size=0.2,random_state=42)
    param = {
        'metric': 'rmse', 
        'random_state': 10,
        'n_estimators': trial.suggest_loguniform('n_estimators', 100, 10000),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.006,0.008,0.01,0.014,0.017,0.02]),
        'max_depth': trial.suggest_categorical('max_depth', [10,20,100]),
        'num_leaves' : trial.suggest_int('num_leaves', 1, 1000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'cat_smooth' : trial.suggest_int('min_data_per_groups', 1, 100)
    }
    model = xgb.XGBRegressor(**param)
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmsle = mean_squared_log_error(test_y, preds,squared=False)
    
    return rmsle

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
params = study.best_trial.params

params

# CatBoost